In [1]:
#Importaciones necesarias
import re  
import json
import nltk
import time
import pickle
import numpy as np

from datetime import datetime
from nltk.stem import SnowballStemmer
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD
from selenium.webdriver.common.by import By
from tensorflow.keras.models import load_model
from selenium.webdriver.common.keys import Keys   
from tensorflow.keras.layers import Dense, Dropout

In [2]:
ignorar = ['!','¡','¿','?','.']
stemmer = SnowballStemmer('spanish')
model = Sequential()

In [3]:
class Func():
    
    def __init__(self)-> None:
        pass
    
    def Json(self, data, filename):
        self.data = data
        archivo = open(filename,'w')
        json.dump(data, archivo, indent=4)
        
    def Modelo(self, mod):        
        self.model = mod
        
    def Final(self, modelo, tags, palabras):
        self.modelo = modelo
        self.tags = tags
        self.palabras = palabras
        
    def cleanEntrada(self, ent_usuario):
        ent_usuario = nltk.word_tokenize(ent_usuario)
        ent_usuario = [stemmer.stem(w.lower()) for w in ent_usuario if w not in ignorar]
        return ent_usuario

    def convVector(self, ent_usuario):    
        ent_usuario = self.cleanEntrada(ent_usuario)    
        vect_entrada = [0]*len(self.palabras) 
        for palabra in ent_usuario:              
            if palabra in self.palabras:          
                indice = self.palabras.index(palabra)
                vect_entrada[indice] = 1                 
        vect_entrada = np.array(vect_entrada)           
        return vect_entrada

    def getTags(self, vect_entrada, LIMITE = 0):
        vect_salida = self.model.predict(np.array([vect_entrada]))[0]
        vect_salida = [[i,r] for i,r in enumerate(vect_salida) if r > LIMITE]
        vect_salida.sort(key=lambda x: x[1], reverse=True)
        print(vect_salida)    
        list_etiquetas = []    
        for r in vect_salida:   
            list_etiquetas.append({"intent": self.tags[r[0]], "probability": str(r[1])})   
        return list_etiquetas

    def getResponse(self, list_etiquetas):
        etiqueta = list_etiquetas[0]['intent']
        list_data = self.data['intents']
        for dat in list_data:
            if etiqueta == dat['tag']:
                list_respuestas = dat['resp']            
                break
        return list_respuestas

    def chatbotRespuesta(self, ent_usuario):
        vect_entrada = self.convVector(ent_usuario)
        list_etiquetas = self.getTags(vect_entrada, LIMITE = 0)
        respuesta = self.getResponse(list_etiquetas)
        return respuesta


In [17]:
obj = Func()
model = Sequential()

class Chat():
    #Creacion del banco de palabras
    dias_pres = ['28/07/2023','30/07/2023']
    dias_virt = ['26/07/2023','27/07/2023','31/07/2023']
    data = { 'intents':[{'tag':'Saludo',
                         'preg':['hola', 'holaa', 'holaaa', 'buenos dias', 'buenas tardes', 'buenas noches', 'buenas'],
                         'resp':['Hola!, soy NabiBot y estoy aqui para ayudarte',
                                 'Selecciona una opcion escribiendo una de las palabras que aparece a continuacion',
                                 '',
                                 '*Menu*',
                                 '*1.* Agendar cita',
                                 '*2.* Servicio de envio',
                                 '*3.* Cotizacion de presupuesto']},
                        
                        {'tag':'Agendar_cita',
                         'preg':['agendar cita', 'programar cita', 'programar una cita', 'reservar una reunion', 'acordar una reunion'],
                         'resp':['*Agendar Cita*',
                                 'Le gustaria que si cita fuese',
                                 '*1.* Virtual',
                                 '*2.* Prescencial']},                         
                        
                        {'tag':'Cita_virtual',
                         'preg':['cita virtual','virtual','reservar cita virtual'],
                         'resp':['*Cita virtual*',
                                 'Para este mes tenemos disponibilidad los dias',
                                 f'{", ".join(dias_virt)}',
                                 '',
                                 '_Todas nuestras citas son a las 3 p.m. por zoom_.',
                                 'Se le hara llegar el link 10 minutos antes el dia de la reunion',
                                 '',
                                 '*Porfavor seleccione una fecha en formato dd/mm/yyyy*']}, 
                        
                        {'tag':'Cita_prescencial',
                         'preg':['cita prescencial','cita presencial','presencial','prescencial'],
                         'resp':['*Cita Prescencial*',
                                 'Para este mes tenemos disponibilidad los siguientes dias',
                                 f'{", ".join(dias_pres)}',
                                 '',
                                 'Porfavor seleccione una fecha en formato dd/mm/yyyy',
                                 '',
                                 'Todas nuestras reuniones presenciales son a las 3 p.m. en nuestras oficinas principales',
                                 'Google Maps: https://goo.gl/maps/rX1cvabSMZhjmPvM9']}, 
                        
                        {'tag':'Servicio_envio',
                         'preg':['servicio de envio','como puedo realizar un envio', 'solicitud de envios','enviar paquetes','como envio un paquete'],
                         'resp':['*Servicio de Envio*',
                                 '*1.* Nacional (Panama)',
                                 '*2.* Internacional (America)',
                                 '*3.* Internacional (Europa, Asia, Oceania)',
                                 '',
                                 'Seleccione el destino del paquete']}, 
                        
                        {'tag':'Nacional_panama',
                         'preg':['nacional'],
                         'resp':['*Nacional (Panama)*'
                                 '*1.* Bocas del Toro',
                                 '*2.* Cocle',
                                 '*3.* Colon',
                                 '*4.* Chiriqui',
                                 '*5.* Darien',
                                 '*6.* Herrera',
                                 '*7.* Los Santos',
                                 '*8.* Panama (Capital)',
                                 '*9.* Veraguas',
                                 '*10.* Panama Oeste',
                                 '*11.* Comarcas',
                                 '',
                                 'Seleccione el destino del paquete']},
                        
                        {'tag':'Bocas_provincia',
                         'preg':['bocas del toro'],
                         'resp':['*Bocas del Toro*',
                                 'Brinde la siguiente informacion del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cedula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},
                        
                        {'tag':'Infor_recibida',
                         'preg':['ameth Cebrian','8-987-2235','1. 15kg'],
                         'resp':['Informacion recibida',
                                 'Gracias por elegir a SIC-Panama']}, 
                        
                        {'tag':'Cotizacion_presupuesto',
                         'preg':['cotizacion ', 'cotizacion del presupuesto', 'cotizar'],
                         'resp':['Hasta la procsima bebi']}, 
                        
                        {'tag':'Despedida',
                         'preg':['Chao', 'Adios', 'Hasta luego'],
                         'resp':['Hasta la procsima bebi']},   
                        
                        {'tag':'Salida',
                         'preg':['exit'],
                         'resp':['sali con exit-o']}, 
                        
                        {'tag':'No_resp',
                         'preg':[''],
                         'resp':['No he logrado entender lo que me escribiste',
                                 '',
                                 'Porfavor revisa lo que has escrito e intenta hacerme la consulta nuevamente']}   
    ]}
    obj.Json(data,'intenciones.json')
    obj.Modelo(model)
    
    palabras = []
    tags = []
    documents = []
        
    for intents in data['intents']:
        tags.append(intents['tag'])
        for preguntas in intents['preg']:
            preguntas_res = nltk.word_tokenize(preguntas)
            palabras.extend(preguntas_res) 
            documents.append((preguntas_res, intents['tag']))
    
    palabras = [stemmer.stem(w.lower()) for w in palabras if w not in ignorar]
    
    pickle.dump(palabras, open('palabras.pkl', 'wb'))
    pickle.dump(tags, open('tags.pkl', 'wb'))
    
    entrenamiento = []
    
    for doc in documents:
        inter = doc[0]
        inter = [stemmer.stem(w.lower()) for w in inter if w not in ignorar]
        ent_cod = []
        for pal in palabras:
            if pal in inter:
                ent_cod.append(1)
            else:
                ent_cod.append(0)
        sal_cod = [0]*len(tags)
        indice = tags.index(doc[1])
        sal_cod[indice] = 1
        entrenamiento.append([ent_cod, sal_cod])
    
    entrenamiento = np.array(entrenamiento, dtype = list)
    x_train = list(entrenamiento[:,0])
    y_train = list(entrenamiento[:,1])
    
    model.add(Dense(128, input_shape=(len(x_train[0]),), activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64,activation = 'relu')) 
    model.add(Dropout(0.5))
    model.add(Dense(len(y_train[0]),activation = 'softmax'))

    sgd = SGD(learning_rate=0.01,momentum=0.9,nesterov=True) 
    model.compile(loss = "categorical_crossentropy",optimizer = sgd, metrics = ["accuracy"])

    hist = model.fit(np.array(x_train),np.array(y_train),epochs = 300,batch_size = 10,verbose = True)
    model.save("modelo_chatbot.h5", hist)
    
    print("##### MODELO CREADO #####")
    
    modelo = load_model('modelo_chatbot.h5')
    data = json.loads(open('intenciones.json').read())
    palabras = pickle.load(open("palabras.pkl","rb"))
    tags = pickle.load(open("tags.pkl","rb"))
    
    obj.Final(modelo, tags, palabras)

Epoch 1/300
4/4 [==============================] - 1s 3ms/step - loss: 2.5765 - accuracy: 0.0541
Epoch 2/300
4/4 [==============================] - 0s 3ms/step - loss: 2.4797 - accuracy: 0.1081
Epoch 3/300
4/4 [==============================] - 0s 3ms/step - loss: 2.3291 - accuracy: 0.2162
Epoch 4/300
4/4 [==============================] - 0s 3ms/step - loss: 2.2903 - accuracy: 0.1622
Epoch 5/300
4/4 [==============================] - 0s 21ms/step - loss: 2.2421 - accuracy: 0.1892
Epoch 6/300
4/4 [==============================] - 0s 11ms/step - loss: 2.0932 - accuracy: 0.4324
Epoch 7/300
4/4 [==============================] - 0s 9ms/step - loss: 2.0611 - accuracy: 0.3514
Epoch 8/300
4/4 [==============================] - 0s 3ms/step - loss: 1.9428 - accuracy: 0.4054
Epoch 9/300
4/4 [==============================] - 0s 7ms/step - loss: 1.7796 - accuracy: 0.5405
Epoch 10/300
4/4 [==============================] - 0s 4ms/step - loss: 1.7041 - accuracy: 0.5405
Epoch 11/300
4/4 [=========

4/4 [==============================] - 0s 5ms/step - loss: 0.2339 - accuracy: 0.9459
Epoch 85/300
4/4 [==============================] - 0s 6ms/step - loss: 0.3169 - accuracy: 0.9189
Epoch 86/300
4/4 [==============================] - 0s 5ms/step - loss: 0.2901 - accuracy: 0.9459
Epoch 87/300
4/4 [==============================] - 0s 4ms/step - loss: 0.3205 - accuracy: 0.9189
Epoch 88/300
4/4 [==============================] - 0s 3ms/step - loss: 0.2948 - accuracy: 0.9730
Epoch 89/300
4/4 [==============================] - 0s 16ms/step - loss: 0.3406 - accuracy: 0.9189
Epoch 90/300
4/4 [==============================] - 0s 4ms/step - loss: 0.3232 - accuracy: 0.9189
Epoch 91/300
4/4 [==============================] - 0s 3ms/step - loss: 0.3614 - accuracy: 0.8649
Epoch 92/300
4/4 [==============================] - 0s 4ms/step - loss: 0.3100 - accuracy: 0.9459
Epoch 93/300
4/4 [==============================] - 0s 8ms/step - loss: 0.3038 - accuracy: 0.9459
Epoch 94/300
4/4 [==============

Epoch 167/300
4/4 [==============================] - 0s 4ms/step - loss: 0.1431 - accuracy: 0.9730
Epoch 168/300
4/4 [==============================] - 0s 16ms/step - loss: 0.0991 - accuracy: 1.0000
Epoch 169/300
4/4 [==============================] - 0s 5ms/step - loss: 0.1187 - accuracy: 0.9730
Epoch 170/300
4/4 [==============================] - 0s 3ms/step - loss: 0.1244 - accuracy: 0.9730
Epoch 171/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0615 - accuracy: 1.0000
Epoch 172/300
4/4 [==============================] - 0s 11ms/step - loss: 0.1815 - accuracy: 0.9459
Epoch 173/300
4/4 [==============================] - 0s 3ms/step - loss: 0.1287 - accuracy: 0.9730
Epoch 174/300
4/4 [==============================] - 0s 4ms/step - loss: 0.1005 - accuracy: 0.9730
Epoch 175/300
4/4 [==============================] - 0s 4ms/step - loss: 0.1637 - accuracy: 0.9459
Epoch 176/300
4/4 [==============================] - 0s 9ms/step - loss: 0.1533 - accuracy: 0.9730
Epoch 17

4/4 [==============================] - 0s 4ms/step - loss: 0.0509 - accuracy: 1.0000
Epoch 250/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0772 - accuracy: 0.9730
Epoch 251/300
4/4 [==============================] - 0s 11ms/step - loss: 0.0326 - accuracy: 1.0000
Epoch 252/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0570 - accuracy: 1.0000
Epoch 253/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0655 - accuracy: 1.0000
Epoch 254/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0713 - accuracy: 1.0000
Epoch 255/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0472 - accuracy: 1.0000
Epoch 256/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0468 - accuracy: 1.0000
Epoch 257/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0925 - accuracy: 0.9459
Epoch 258/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0631 - accuracy: 1.0000
Epoch 259/300
4/4 [====

In [5]:
#Importacion de selenium para abrir el software automatizado en Edge
from selenium import webdriver
driver = webdriver.Edge()
driver.get("https://web.whatsapp.com/")

In [18]:
def Verificar(usuario):
    try:
        num_mens = usuario.find_element(By.CLASS_NAME,"_1pJ9J").text
        mens_leer = re.findall('\d+' ,num_mens)
        if len(mens_leer) != 0:
            pendiente = True
        else:
            pendiente = False
    except:
        pendiente = False
    return pendiente

def obtenerMensaje(chat):
    mensaje = chat.find_element(By.CLASS_NAME,"_21Ahp").text
    html = chat.get_attribute('outerHTML')
    persona = re.findall(r'data-pre-plain-text=.{1}\[.+].(.+?):', html)
    return persona, mensaje

usuario_men = ''

while usuario_men != 'salir':
    print('#', end = " ")
    time.sleep(1)
    conversaciones = driver.find_elements(By.CLASS_NAME, "_1Oe6M") 
    
    for usuario in conversaciones:
        nombres = usuario.find_element(By.CLASS_NAME,"_21S-L")
        no_resp = Verificar(usuario)
        
        if no_resp:
            conversacion = usuario.find_element(By.CLASS_NAME,"vQ0w7")
            conversacion.click()  
            time.sleep(1)        
            chats = driver.find_elements(By.CLASS_NAME,"_27K43")     
  
            for chat in chats[-4:]:
                persona, mensaje = obtenerMensaje(chat)
                usuario_men = mensaje.lower()                     
                
            print("Usuario:",persona, "Mensaje:", usuario_men, end = " ")
            respuesta = driver.find_element(By.CLASS_NAME,"_3Uu1_")              
            bot_resp = obj.chatbotRespuesta(usuario_men)        
            
            for res in bot_resp:
                respuesta.send_keys(res, Keys.SHIFT + Keys.ENTER)          
            respuesta.send_keys(Keys.ENTER)                 
            time.sleep(1)

            webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()    

# Usuario: ['Luis Leiva'] Mensaje: agendar cita [[1, 0.72984743], [5, 0.08318617], [7, 0.055283904], [10, 0.041272044], [11, 0.033145446], [6, 0.028920066], [8, 0.011886961], [4, 0.005288999], [9, 0.005239141], [2, 0.0025630048], [0, 0.0019759748], [3, 0.0013908647]]
# # # # # # # # Usuario: ['Luis Leiva'] Mensaje: virtual [[2, 0.7236088], [11, 0.1761207], [5, 0.021196788], [7, 0.01750371], [6, 0.016857516], [0, 0.012393597], [9, 0.009808528], [10, 0.008826126], [1, 0.0059668445], [3, 0.0059639965], [8, 0.0010582659], [4, 0.0006950915]]
# # # # # # Usuario: ['Luis Leiva'] Mensaje: 27/07/2023 [[11, 0.789274], [5, 0.057740398], [0, 0.03743865], [7, 0.034161657], [9, 0.02271039], [10, 0.01691144], [8, 0.014962903], [3, 0.010499032], [6, 0.009684118], [2, 0.0033049558], [1, 0.0019746448], [4, 0.0013379055]]
# # # # # # # # # # # # # # Usuario: ['Luis Leiva'] Mensaje: servicio de envio [[4, 0.79824704], [7, 0.091954894], [11, 0.03221595], [5, 0.03122949], [9, 0.013403603], [6, 0.007428176],

KeyboardInterrupt: 